In [1]:
#Script to get metadata and fields from hero website for documents in pfas_150_sem_citations

#First lets import the modules that will be needed. Pandas (dataframes and excel sheets), Selenium(webrowser)
#Numpy(basic math things), Time (courtesy sleeps for web browser)


import pandas as pd
import numpy as np
import time as t
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

In [2]:
#Load in a clowder log excel spreadsheet
df = pd.read_excel(r"L:\PRIV\ToxValDB\Upload to Clowder\source_pfas_150_sem\clowder_log_source_pfas_150_sem_20220921.xlsx")

In [6]:
#Initiate the web browser options and url
hero_url = 'https://hero.epa.gov/hero/index.cfm/reference/details/reference_id/'
Options = webdriver.ChromeOptions()
Options.headless = True
s=Service(r'C:\Users\mmille16\chromedriver.exe')
driver = webdriver.Chrome(service=s, options = Options)

for j in range(1,len(df)):
    name = df.loc[j,'subDir1']
    hero_id = str(name.replace('HEROID','').replace('.pdf',''))
    t.sleep(1)
    driver.get(hero_url + hero_id)
    labels = driver.find_elements(By.CLASS_NAME, 'label')
    fields = driver.find_elements(By.CLASS_NAME, 'field')
    for i in range(len(labels)):
        raw = labels[i].text
        if fields[i].get_attribute('id') == 'abstract_div':
            continue
        text = fields[i].text
        metadata_start = text.index(raw) + len(raw) +1
        metadata = text[metadata_start:]
        if raw == 'URL':
            metadata = metadata.replace('EXIT','')
        df.loc[j,raw] = metadata
df = df.fillna('N/A')
df = df.drop("", axis = 1)

In [8]:
#Save the final metadata dataframe to an excel file. While I need to upload the pdfs to clowder first and run this again
#it is helpful to look at the metadata in the spreadsheet to check for errors

out_path = r"L:\PRIV\ToxValDB\Upload to Clowder\source_pfas_150_sem\pfas_150_sem_clowder_log_metadata_10032022.xlsx"
with pd.ExcelWriter(out_path) as writer:
    df.to_excel(writer, index = False)

Testing ideas below here. Above is the final script to create the metadata spreadsheets and save them as xlsx files

In [36]:
#lets test some ideas for just one document first. The goals for the script are just to add columns to df based on the metadata fields
#Then populate those fields for a certain document.

#Note: Not all documents have the same metadata fields. At the end of the script we will look for empty cells an populate with N/A
#Note: Some documents have the abstract and other larger metadata fields we do not want. We will stop once we hit the word 'abstract' as a field


#Initiate the web browser options and url
hero_url = 'https://hero.epa.gov/hero/index.cfm/reference/details/reference_id/'
Options = webdriver.ChromeOptions()
Options.headless = True
s=Service(r'C:\Users\mmille16\Documents\chromedriver.exe')
driver = webdriver.Chrome(service=s, options = Options)

#Select a hero id from the hero id column
j = 12
name = df.loc[j,'subDir1']
hero_id = str(name.replace('HEROID','').replace('.pdf',''))
driver.get(hero_url + hero_id)
t.sleep(1)

#Check in on the dataframe and find the list of its current metadata fields
headers = []
check_headers = [] #in case of capitalization differences: we will populate this with the metadata fields in lower case

#Get the list of dataframe headers
col = df.columns
for column in col:
    headers.append(column)

#Populate the list of headers in lower case
for item in headers:
    check_headers.append(item.lower())
    
#Find all the metadata fields titles
labels = driver.find_elements(By.CLASS_NAME, 'label')

#Find the actual contents of the metadata
values = driver.find_elements(By.CLASS_NAME,'field')

#Look through all the metadata fields
for i in range(len(labels)):
    raw = labels[i].text #Bolded titles of metadata fields
    header = raw.lower() #Put the titles in lower case. Just in case there are differences in capitalization
    if values[i].get_attribute('id') == 'abstract_div': #skip metadata in the abstract section
        continue
    text = values[i].text
    search_text = text.lower()
    metadata_start = search_text.rindex(header) + len(header) + 1
    metadata = text[metadata_start:]
    if header == 'url':
        metadata = metadata.replace('EXIT','')
    df.loc[j,raw] = metadata
#Close the browser once we are done
driver.quit()

In [47]:
df.loc[[1]]

,origPath,type,rootDir,relPath,maxFolder,subDir0,subDir1,parentPath,parentId,parentType,uuid
1,L:\PRIV\ToxValDB\Upload to Clowder\To Do\sourc...,folder,L:\PRIV\ToxValDB\Upload to Clowder\To Do\sourc...,PFAS 150 SEM,0,PFAS 150 SEM,NaN,PFAS 150 SEM,5e31dc1e99323f93a9f5cec0,dataset,62f6885ae4b055edffc335df


In [38]:
df = df.fillna('N/A')
df = df.drop("", axis = 1)

In [42]:
df.iloc[0,0]

'L:\\PRIV\\ToxValDB\\Upload to Clowder\\To Do\\source_pfas_150_sem\\source_pfas_150_sem\\PFAS 150 SEM'

In [40]:
for col in df.columns:
    print(col)

origPath
type
rootDir
relPath
maxFolder
subDir0
subDir1
parentPath
parentId
parentType
uuid
HERO ID
Reference Type
Title
Author(s)
Year
Is Peer Reviewed?
Journal
Volume
Issue
Page Numbers
Language
PMID
DOI
Web of Science Id
Relationship(s)
Keywords
